In [11]:
import hashlib
import random
import rsa
from sklearn import datasets

class Block:
    def __init__(self, transaction, previous_hash):
        self.transaction = transaction
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()
        print(f"\nBlock created with hash {self.hash} and previous hash {self.previous_hash}\n")

    def calculate_hash(self):
        return hashlib.sha256(str(self.transaction).encode() + str(self.previous_hash).encode() + str(self.nonce).encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self._chain = [self.create_genesis_block()]

    @property
    def chain(self):
        return self._chain.copy()

    def create_genesis_block(self):
        print("\nCreating genesis block...\n")
        return Block("\nGenesis Block", "0\n")

    def proof_of_work(self, block, difficulty=3):
        prefix = '0' * difficulty
        while not block.hash.startswith(prefix):
            block.nonce += 1
            block.hash = block.calculate_hash()
        print(f"\nProof of work completed for block with nonce {block.nonce} and hash {block.hash}\n")
        return block

    def add_block(self, transaction):
        new_block = Block(transaction, self.chain[-1].hash)
        self.proof_of_work(new_block)
        self._chain.append(new_block)
        print(f"\nBlock added to the chain with hash {new_block.hash}\n")

    def is_chain_valid(self, chain):
        for i in range(1, len(chain)):
            current_block = chain[i]
            prev_block = chain[i - 1]

            if current_block.hash != current_block.calculate_hash():
                print("\nCurrent block hash does not match calculated hash.\n")
                return False

            if current_block.previous_hash != prev_block.hash:
                print("\nCurrent block's previous hash does not match the previous block's hash.\n")
                return False

        return True

def is_authorized(pub_key):
    return pub_key in [hospitalA.public_key, hospitalB.public_key]

class Hospital:
    def __init__(self, name, hospital_id):
        self._name = name
        self._id = hospital_id
        self.public_key, self.private_key = rsa.newkeys(2048)
        self._blockchain = Blockchain()
        self._ip_address = f"{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}"
        print(f"\nInitializing {self._name} (ID: {self._id}, IP: {self._ip_address}). Keys generated.\n")

    @property
    def blockchain(self):
        return self._blockchain

    def verify_requester(self, pub_key, requester_id):
        return is_authorized(pub_key) and requester_id in ["A", "B"]

    def create_transaction(self, recipient_pub_key, data_entry):
        encrypted_data = rsa.encrypt(data_entry.tobytes(), recipient_pub_key)
        signature = rsa.sign(encrypted_data, self.private_key, "SHA-256")
        print(f"\nTransaction created and signed by {self._name}.\n")
        return {"data": encrypted_data, "signature": signature}

    def verify_and_add_data(self, sender_hospital_obj, transaction):
        data = transaction["data"]
        signature = transaction["signature"]

        if not is_authorized(sender_hospital_obj):
            print(f"\n{self._name} failed to verify sender.\n")
            return

        if rsa.verify(data, signature, sender_hospital_obj.public_key):
            decrypted_data = rsa.decrypt(data, self.private_key)
            self._blockchain.add_block(decrypted_data)
            print(f"\nData verified and added to {self._name}'s blockchain.\n")
        else:
            print(f"\n{self._name} failed to verify signature.\n")

    def request_data(self, recipient_pub_key, row_indices):
        encrypted_request = rsa.encrypt(str(row_indices).encode(), recipient_pub_key)
        signature = rsa.sign(encrypted_request, self.private_key, "SHA-256")
        print(f"\n{self._name} created a data request.\n")
        return {"\nrequest": encrypted_request, "signature": signature}


    def grant_data_request(self, sender_pub_key, request_transaction):
        encrypted_request = request_transaction["request"]
        signature = request_transaction["signature"]

        if not is_authorized(sender_pub_key):
            print(f"\n{self._name} failed to verify requester's identity.\n")
            return

        if rsa.verify(encrypted_request, signature, sender_pub_key):
            row_indices = eval(rsa.decrypt(encrypted_request, self.private_key).decode())
            if not all(isinstance(i, int) and 0 <= i < data_size for i in row_indices):
                print(f"\n{self._name} found an invalid index in the request.\n")
                return None
            requested_data = data[row_indices]
            print(f"\n{self._name} granted the data request.\n")
            return self.create_transaction(sender_pub_key, requested_data)
        else:
            print(f"\n{self._name} failed to verify the requester's signature.\n")

    def replace_chain(self, new_chain):
        if len(new_chain) > len(self._blockchain.chain) and self._blockchain.is_chain_valid(new_chain):
            self._blockchain._chain = new_chain
            print(f"\n{self._name} replaced its blockchain with a new one.\n")

class HospitalC:
    def __init__(self):
        self.public_key, self.private_key = rsa.newkeys(2048)
        self._blockchain = Blockchain()

    def modify_block_data(self,block_index, new_data):
        if 0 <= block_index < len(self._blockchain.chain):
            block_to_modify = self._blockchain.chain[block_index]
            block_to_modify.transaction = new_data
            block_to_modify.hash = block_to_modify.calculate_hash()
            print(f"\nHospitalC modified data in block {block_index}\n")
        else:
            print(f"\nHospitalC could not modify data in block {block_index}\n")

    def broadcast_modified_chain(self, target_hospital):
        if target_hospital.verify_requester(self.public_key, 'C') and \
                not target_hospital.blockchain.is_chain_valid(self._blockchain.chain):
            target_hospital.replace_chain(self._blockchain.chain)
            print("\nHospitalC broadcasted modified chain to target hospital.\n")
        else:
            print(f"\nHospitalC could not broadcast modified chain to target hospital.\n")


    def add_data_to_blockchain(self, data_entry):
        encrypted_data = rsa.encrypt(data_entry.tobytes(), self.public_key)
        self._blockchain.add_block(encrypted_data)


# Demo/Test
data_size = 10
data = datasets.make_regression(n_samples=data_size, n_features=3)[0]

hospitalA = Hospital("Hospital A", "A")
hospitalB = Hospital("Hospital B", "B")
hospitalC = HospitalC()

# Let's say hospitalA wants to send some data to hospitalB
transaction = hospitalA.create_transaction(hospitalB.public_key, data[0])
hospitalB.verify_and_add_data(hospitalA, transaction)

# HospitalC tries to tamper with HospitalB's data
hospitalC.modify_block_data(1, "\nTampered Data")
hospitalC.broadcast_modified_chain(hospitalB)

# Check if HospitalB's blockchain data has been tampered with or not
if not hospitalB.blockchain.is_chain_valid(hospitalB.blockchain.chain):
    print("\nHospital B's blockchain has been tampered with!")
else:
    print("\nHospital B's blockchain is valid.")



Creating genesis block...


Block created with hash 1585162ef021be51d1d3f813ba71f87f13095fdf36d49fed131f874dedd2ca3a and previous hash 0



Initializing Hospital A (ID: A, IP: 93.109.16.106). Keys generated.


Creating genesis block...


Block created with hash 1585162ef021be51d1d3f813ba71f87f13095fdf36d49fed131f874dedd2ca3a and previous hash 0



Initializing Hospital B (ID: B, IP: 238.0.60.82). Keys generated.


Creating genesis block...


Block created with hash 1585162ef021be51d1d3f813ba71f87f13095fdf36d49fed131f874dedd2ca3a and previous hash 0



Transaction created and signed by Hospital A.


Hospital B failed to verify sender.


HospitalC could not modify data in block 1


HospitalC could not broadcast modified chain to target hospital.



AttributeError: 'Hospital' object has no attribute 'blockchain'